In [1]:
%%html
<style>   
div.input {   
    display:none;   
}
/style>

In [2]:
from pat20 import pat

In [3]:
from i2c_client import i2c
from sysmon_client import sysmon
import time
import threading
from IPython.display import display, clear_output,HTML
from ipywidgets.widgets import Layout, HBox, VBox, Label, AppLayout, Button, Output, Dropdown, FloatProgress, Text
import numpy as np
import bqplot as bq
HIST_VALUES=60
LOCK = threading.Lock()

In [4]:
x = np.arange(-HIST_VALUES, 1, 1.0,  dtype=float)
y = np.zeros(HIST_VALUES+1, dtype=float)

x_lpd = np.arange(-HIST_VALUES, 1, 1.0,  dtype=float)
y_lpd = np.zeros(HIST_VALUES+1, dtype=float)

x_pld = np.arange(-HIST_VALUES, 1, 1.0,  dtype=float)
y_pld = np.zeros(HIST_VALUES+1, dtype=float)

x_full = np.arange(-HIST_VALUES, 1, 1.0,  dtype=float)
y_full = np.zeros(HIST_VALUES+1, dtype=float)


In [5]:
i2c.SetIpAndPort("127.0.0.1", 9090)
sysmon.SetIpAndPort("127.0.0.1", 9090)


In [6]:
ret = i2c.XI2c_Initialize(2)


In [7]:
# Plot Elements

# scales
x_sc = bq.LinearScale(min=-HIST_VALUES, max = 10)
#y_sc = bq.LinearScale(min=0, max=10000)
y_sc = bq.LinearScale(min = 0)

# axis
x_ax = bq.Axis(
     label="secs",
     scale=x_sc
)

y_ax = bq.Axis(
     label="mW",
     scale=y_sc,
     orientation="vertical"
)



In [8]:
# Lines
Line = bq.Lines(
     x=x,
     y=y,
     name="fpd",
     scales={"x": x_sc, "y": y_sc}
)

Line_lpd = bq.Lines(
     x=x_lpd,
     y=y_lpd,
     name="lpd",
     colors=["red"],
     scales={"x": x_sc, "y": y_sc}
)

Line_pld = bq.Lines(
     x=x_pld,
     y=y_pld,
     name="pld",
     colors=["green"],
     scales={"x": x_sc, "y": y_sc}
)

Line_full = bq.Lines(
     x=x_full,
     y=y_full,
     name="Total",
     colors=["black"],
     scales={"x": x_sc, "y": y_sc}
)

bulk_lables = ["" for bindex in range(HIST_VALUES)]

test_label = bq.Label(x=x_pld, y=y_pld, text=bulk_lables, default_size=10, scales={"x": x_sc, "y": y_sc},
                  font_weight='bolder', colors=['green'], update_on_move=True)

lpd_label = bq.Label(x=x_lpd, y=y_lpd, text=bulk_lables, default_size=10, scales={"x": x_sc, "y": y_sc},
                  font_weight='bolder', colors=['red'], update_on_move=True)


fpd_label = bq.Label(x=x, y=y, text=bulk_lables, default_size=10, scales={"x": x_sc, "y": y_sc},
                  font_weight='bolder', colors=['blue'], update_on_move=True)

full_label = bq.Label(x=x_full, y=y_full, text=bulk_lables, default_size=10, scales={"x": x_sc, "y": y_sc},
                  font_weight='bolder', colors=['black'], update_on_move=True)


In [9]:
pw = Output()

# progress bar and label
w1 = FloatProgress(
   value=y_full[-1],
   min=0,
   max=20000,
   description="TOT:",
   style={"description_width": "initial"},
   layout=Layout(width="365px")
)

w2 = Label(
   value= str(np.round(y_full[-1],2)) + " (mW)",
   layout=Layout(margin="0 10px 0 31px")
)


w12 = HBox(
    children=(w1, w2),
    layout=Layout(margin="0 0 0 43px")
)

# progress bar and label
w1pld = FloatProgress(
   value=y_pld[-1],
   min=0,
   max=20000,
   description="PLD:",
   style={"description_width": "initial"},
   layout=Layout(width="365px")
)

w2pld = Label(
   value= str(np.round(y_pld[-1],2)) + " (mW)",
   layout=Layout(margin="0 10px 0 31px")
)


w12pld = HBox(
    children=(w1pld, w2pld),
    layout=Layout(margin="0 0 0 43px")
)

# progress bar and label
w1lpd = FloatProgress(
   value=y_lpd[-1],
   min=0,
   max=20000,
   description="LPD:",
   style={"description_width": "initial"},
   layout=Layout(width="365px")
)

w2lpd = Label(
   value= str(np.round(y_lpd[-1],2)) + " (mW)",
   layout=Layout(margin="0 10px 0 31px")
)


w12lpd = HBox(
    children=(w1lpd, w2lpd),
    layout=Layout(margin="0 0 0 43px")
)

# progress bar and label
w1fpd = FloatProgress(
   value=y[-1],
   min=0,
   max=20000,
   description="FPD:",
   style={"description_width": "initial"},
   layout=Layout(width="365px")
)

w2fpd = Label(
   value= str(np.round(y[-1],2)) + " (mW)",
   layout=Layout(margin="0 10px 0 31px")
)


w12fpd = HBox(
    children=(w1fpd, w2fpd),
    layout=Layout(margin="0 0 0 43px")
)

box=VBox(
        children=(pw, w12, w12pld, w12lpd, w12fpd),
        layout=Layout( width="600px")
)

temp = Output()
tempStr = "PS {0:3.1f}° C".format(float(pat.psTemp))
tempLabel = "<h1 style='font-size:20px'>" + tempStr +  "</h1>"
tempLabel = HTML(tempLabel)
#with temp:
#    display(tempLabel)

box_layout = Layout(display='flex',
    flex_flow='row',
    align_items='center',
    width='95%',
    border="solid 2px gray",
    margin="0 0 0 43px")

hbox = HBox(
    children=(box, temp),
    layout = box_layout
    #layout=Layout(border="solid 2px gray", margin="0 0 0 43px")
)


In [10]:
fig = bq.Figure(
    layout=Layout(width="800px", height="750px"),
    marks=[Line, Line_lpd, Line_pld, Line_full, test_label, lpd_label, fpd_label, full_label],
    axes=[x_ax, y_ax],
    stroke_width=1,
    interpolation='monotone',
    fig_margin=dict(top=10, bottom=40, left=50, right=10)
)
#def create_expanded_button(description, button_style):
#    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

menu = Dropdown(
       options=['RAILS', 'GRAPH', 'SYSMON'],
       value='GRAPH',
       description='Display:')

menu_box=HBox(
        children=(menu,),
        layout=Layout(align_items='center')
)

toggle = 'GRAPH'

#ToggleOutput = create_expanded_button("HTML", 'info')
out = Output()

al = AppLayout(header=hbox,
    left_sidebar=None,
    center=menu_box,
    right_sidebar= None,
    footer=fig,
    border="solid 2px gray",
    margin="0 0 0 43px",
    pane_widths=[0, 3, 0],
    pane_heights=['200px', '60px', '800px'])

def on_change(change):
    global toggle
    global al
    LOCK.acquire(True)
#        clear_output()
    if change['type'] == 'change' and change['name'] == 'value' and change['new'] == 'RAILS':
        al.footer = out
        toggle = 'RAILS'
    elif change['type'] == 'change' and change['name'] == 'value' and change['new'] == 'GRAPH':
        al.footer = fig
        toggle = 'GRAPH'
    elif change['type'] == 'change' and change['name'] == 'value' and change['new'] == 'SYSMON':
        al.footer = out
        toggle = 'SYSMON'
    out.clear_output(wait=True)
    LOCK.release()

menu.observe(on_change)


In [11]:
def work():
    global x
    global y
    global x_lpd
    global y_lpd
    global x_pld
    global y_pld
    global x_full
    global y_full
    global toggle
    global out
    #global ToggleOutput
    global al
    global temp
    while True:
        #pat.PullINA226Data(i2c)
        #pat.PullSysmonData(sysmon)
        pat.PullData(i2c, sysmon)
        str0 = pat.generateFullReport()
        str1 = pat.generateFullSysmonReport()
        y = np.append(y, pat.fpdPower/1000)
        if(y.shape[0] > HIST_VALUES+1):
            y = y[1:HIST_VALUES+2]
        y_lpd = np.append(y_lpd, pat.lpdPower/1000)
        if(y_lpd.shape[0] > HIST_VALUES+1):
            y_lpd = y_lpd[1:HIST_VALUES+2]
        y_pld = np.append(y_pld, pat.pldPower/1000)
        if(y_pld.shape[0] > HIST_VALUES+1):
            y_pld = y_pld[1:HIST_VALUES+2]
        y_full = np.append(y_full, pat.fpdPower/1000 + pat.lpdPower/1000 + pat.pldPower/1000)
        if(y_full.shape[0] > HIST_VALUES+1):
            y_full = y_full[1:HIST_VALUES+2]
        LOCK.acquire(True)
        # update plot
        Line.x = x
        Line.y = y
        Line.labels=['fpd {: 4.1f}'.format(y[HIST_VALUES])]
        fpd_label.x = x
        fpd_label.y = y
        fpd_label.text = bulk_lables + ['..FPD {: 4.1f}'.format(y[HIST_VALUES])]

        Line_lpd.x = x_lpd
        Line_lpd.y = y_lpd
        Line_lpd.labels=['lpd {: 4.1f}'.format(y_lpd[HIST_VALUES])]
        lpd_label.x = x_lpd
        lpd_label.y = y_lpd
        lpd_label.text = bulk_lables + ['..LPD {: 4.1f}'.format(y_lpd[HIST_VALUES])]

        Line_pld.x = x_pld
        Line_pld.y = y_pld
        Line_pld.labels=['  pld {: 4.1f}'.format(y_pld[HIST_VALUES])]
        test_label.x = x_pld
        test_label.y = y_pld
        test_label.text = bulk_lables + ['..PLD {: 4.1f}'.format(y_pld[HIST_VALUES])]

        Line_full.x = x_full
        Line_full.y = y_full
        full_label.x = x_full
        full_label.y = y_full
        full_label.text = bulk_lables + ['..TOTAL {: 4.1f}'.format(y_full[HIST_VALUES])]

        if toggle == "RAILS":
            #display(y)
            html_text = HTML(str0)
            with al.footer:
                display(html_text)
        if toggle == "SYSMON":
            #display(y)
            html_text_sys = HTML(str1)
            with al.footer:
                display(html_text_sys)

        tempStr = "PS {0:3.1f}° C".format(float(pat.psTemp))
        tempLabel = "<h1 style='font-size:50px'>" + tempStr +  "</h1>"
        tempLabel = HTML(tempLabel)
        with temp:
            #clear_output(wait=True)
            display(tempLabel)

        time.sleep(1)
        if toggle == "RAILS":
            out.clear_output(wait=True)
        elif toggle == "SYSMON":
            out.clear_output(wait=True)
        # Update progress bar and label
        w1.value = y_full[-1]
        w2.value = str(np.round(y_full[-1],2)) + " (mW)"

        w1pld.value = y_pld[-1]
        w2pld.value = str(np.round(y_pld[-1],2)) + " (mW)"

        w1lpd.value = y_lpd[-1]
        w2lpd.value = str(np.round(y_lpd[-1],2)) + " (mW)"

        w1fpd.value = y[-1]
        w2fpd.value = str(np.round(y[-1],2)) + " (mW)"
        with temp:
            clear_output(wait=True)
        LOCK.release()

# create Thread
thread = threading.Thread(target=work)
#work()

# start thread
thread.start()
#thread.join()


In [12]:
display(al)


AppLayout(children=(HBox(children=(VBox(children=(Output(), HBox(children=(FloatProgress(value=0.0, descriptio…